## 0) 不要省略异常信息的记录

很多时候，由于疏忽大意，开发者很容易捕获了异常却没有记录异常信息，导致程序上线后真的出现了问题却没有记录可查。

```java
public void logAnException() {
    try {
    } catch (NumberFormatException e) {
        log.error("哦，错误竟然发生了: " + e);
    }
}
```

## 1) 尽量不要捕获 RuntimeException

> 尽量不要 catch RuntimeException，比如 NullPointerException、IndexOutOfBoundsException 等等，应该用预检查的方式来规避。

如果有些异常预检查不出来, 比如NumberFormatException也属于 RuntimeException，但没办法预检查，所以就应该用 catch 捕获处理.

## 2) 尽量使用 try-with-resource 来关闭资源

当需要关闭资源时，尽量不要使用 try-catch-finally，禁止在 try 块中直接关闭资源.

```java
// 反例
public void doNotCloseResourceInTry() {
    FileInputStream inputStream = null;
    try {
        File file = new File("./tmp.txt");
        inputStream = new FileInputStream(file);
        inputStream.close();
    } catch (FileNotFoundException e) {
        log.error(e);
    } catch (IOException e) {
        log.error(e);
    }
}
```

原因也很简单，因为一旦 `close()` 之前发生了异常，那么资源就无法关闭。直接使用 **try-with-resource**来处理是最佳方式. 

```java
public void automaticallyCloseResource() {
    File file = new File("./tmp.txt");
    try (FileInputStream inputStream = new FileInputStream(file); {
    } catch (FileNotFoundException e) {
        log.error(e);
    } catch (IOException e) {
        log.error(e);
    }
}
```

如果资源没有实现 AutoCloseable 接口, 就在 finally 块关闭流:

```java
public void closeResourceInFinally() {
    FileInputStream inputStream = null;
    try {
        File file = new File("./tmp.txt");
        inputStream = new FileInputStream(file);
    } catch (FileNotFoundException e) {
        log.error(e);
    } finally {
        if (inputStream != null) {
            try {
                inputStream.close();
            } catch (IOException e) {
                log.error(e);
            }
        }
    }
}
```

## 3) 不要在 finally 块中使用 return

> try 块中的 return 语句执行成功后，并不会马上返回，而是继续执行 finally 块中的语句，如果 finally 块中也存在 return 语句，那么 try 块中的 return 就将被覆盖。

```java
private int x = 0;
public int checkReturn() {
    try {
        return ++x;
    } finally {
        return ++x;
    }
}
```

## 4) 捕获具体的子类而不是捕获 Exception 类

```java
// 反例
try {
   someMethod();
} catch (Exception e) {
   LOGGER.error("method has failed", e);
}
```



## 5) finally 块中永远不要抛出任何异常

```java
try {
  someMethod();  //Throws IndexException
} finally {
  cleanUp();    //如果finally还抛出异常，那么IndexException将永远丢失
}
```

只要 `cleanUp()` 永远不会抛出任何异常，上面的代码没有问题，但是如果 `someMethod()` 抛出一个异常，并且在 finally 块中，`cleanUp()` 也抛出另一个异常，那么程序只会把第二个异常抛出来，原来的第一个异常（正确的原因）将永远丢失。如果在 finally 块中调用的代码可能会引发异常，请确保要么处理它，要么将其记录下来。永远不要让它从 finally 块中抛出来。

## 6) 对于不打算处理的异常，直接使用 finally

```java
try {
  someMethod();  //Method 2
} finally {
  cleanUp();    //do cleanup here
}
```

这是一个很好的做法，如果在你的方法中你正在访问 Method 2，而 Method 2 抛出一些你不想在 Method 1 中处理的异常，但是仍然希望在发生异常时进行一些清理，然后在 finally 块中进行清理，不要使用 catch 块。

## 7) 记住早 throw 晚 catch 原则


这可能是关于异常处理最著名的原则，简单说，应该尽快抛出(throw)异常，并尽可能晚地捕获(catch)它。应该等到有足够的信息来妥善处理它。

这个原则隐含地说，你将更有可能把它放在低级方法中，在那里你将检查单个值是否为空或不适合。而且你会让异常堆栈跟踪上升好几个级别，直到达到足够的抽象级别才能处理问题。